# Convolutional Neural Network on pixel neighborhoods

This notebook reads the pixel-neighborhood data written out by the Dataflow program of [1_explore.ipynb](./1_explore.ipynb) and trains a simple convnet model on Cloud ML Engine.


In [1]:
BUCKET = 'cloud-training-demos-ml'
PROJECT = 'cloud-training-demos'
REGION = 'us-central1'

In [2]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [3]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

Updated property [core/project].
Updated property [compute/region].


In [ ]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

## Train CNN model locally

In [ ]:
%%bash
OUTDIR=${PWD}/cnn_trained
DATADIR=${PWD}/preproc/tfrecord
rm -rf $OUTDIR
gcloud ml-engine local train \
    --module-name=trainer.train_cnn --package-path=${PWD}/ltgpred/trainer \
    -- \
    --train_steps=5 --num_eval_records=512 --train_batch_size=1 --num_cores=1 \
    --job-dir=$OUTDIR --train_data_path=${DATADIR}/train* --eval_data_path=${DATADIR}/eval*

## Training lighting prediction model on CMLE using GPU

Let's train on a machine with 4 GPUs.

In [12]:
%writefile largemachine.yaml
trainingInput:
  scaleTier: CUSTOM
  masterType: complex_model_m_gpu

Overwriting largemachine.yaml


In [6]:
%%bash
OUTDIR=gs://${BUCKET}/lightning/cnn_trained_gpu
DATADIR=gs://$BUCKET/lightning/preproc/tfrecord
JOBNAME=ltgpred_cnn_$(date -u +%y%m%d_%H%M%S)
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --module-name=trainer.train_cnn --package-path=${PWD}/ltgpred/trainer --job-dir=$OUTDIR \
    --region=${REGION} --scale-tier=CUSTOM --config=largemachine.yaml \
    --python-version=3.5 --runtime-version=1.8 \
    -- \
    --train_data_path=${DATADIR}/train-* --eval_data_path=${DATADIR}/eval* \
    --train_steps=10000 --train_batch_size=256 \
    --num_eval_records=128000 --nlayers=5 --dprob=0.05 --ksize=3 --nfil=100

jobId: ltgpred_cnn_180926_212917
state: QUEUED


Removing gs://cloud-training-demos-ml/lightning/cnn_trained_gpu/packages/12c1a2acd7230b7448e2a1bbb90d632711e0cafc87df2e6e14fbfc2371ffb0f6/ltgpred-0.0.1.tar.gz#1537983488310425...
/ [1/1 objects] 100% Done                                                       
Operation completed over 1 objects.                                              
Job [ltgpred_cnn_180926_212917] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ml-engine jobs describe ltgpred_cnn_180926_212917

or continue streaming the logs with the command

  $ gcloud ml-engine jobs stream-logs ltgpred_cnn_180926_212917


The training completed after 20 minutes with this result:
<pre>
Eval results at step 10000: {'global_step': 10000, 'rmse': 0.49927762, 'accuracy': 0.6623125, 'loss': 0.6917048}
</pre>
Training longer (100,000 steps, for 2 hours) didn't seem to do much:
<pre>
Eval results at step 100000: {'loss': 0.6889524, 'accuracy': 0.6721641, 'global_step': 100000, 'rmse': 0.4978987}
</pre>
Increasing the number of layers from 3 to 5 and lowering the dropout to 0.05 didn't help either.

## Training lightning prediction model on CMLE using TPUs

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/lightning/cnn_trained
DATADIR=gs://$BUCKET/lightning/preproc/tfrecord
JOBNAME=ltgpred_cnn_$(date -u +%y%m%d_%H%M%S)
gsutil -m rm -rf $OUTDIR
gcloud ml-engine jobs submit training $JOBNAME \
    --module-name=trainer.train_cnn --package-path=${PWD}/ltgpred/trainer --job-dir=$OUTDIR \
    --region=${REGION} --scale-tier=BASIC_TPU \
    --python-version=3.5 --runtime-version=1.8 \
    -- \
    --train_data_path=${DATADIR}/train* --eval_data_path=${DATADIR}/eval* \
    --train_steps=10000 --train_batch_size=1024 \
    --num_eval_records=128000 --nlayers=0 --num_cores=32 --use_tpu

# When I ran it, training finished with accuracy=???


Copyright 2018 Google Inc. Licensed under the Apache License, Version 2.0 (the \"License\"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an \"AS IS\" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License